based on [crash course code example](https://wiki.osarch.org/index.php?title=IfcOpenShell_code_examples#Crash_course)


In [1]:
ifcPath="../data/Duplex_A.ifc"

# Reading IFC

In [2]:
import ifcopenshell
ifc = ifcopenshell.open(ifcPath)

Let's see what IFC schema we are using:

In [3]:
print(ifc.schema)

IFC2X3


Let's get the first piece of data in our IFC file:

In [4]:
print(ifc.by_id(1))

#1=IfcOrganization($,'Autodesk Revit Architecture 2011',$,$,$)


But getting data from beginning to end isn't too meaningful to humans. What if we knew a GlobalId value instead?

In [5]:
print(ifc.by_guid('2O2Fr$t4X7Zf8NOew3FNld')) #guid - global unique identifyer

#4131=IfcWallStandardCase('2O2Fr$t4X7Zf8NOew3FNld',#33,'Basic Wall:Interior - Partition (92mm Stud):138584',$,'Basic Wall:Interior - Partition (92mm Stud):128360',#4118,#4130,'138584')


If we're not looking specifically for a single element, perhaps let's see how many walls are in our file, and count them:

In [6]:
walls = ifc.by_type('IfcWall')
print(len(walls))

57


wall = ifc.by_type('IfcWall')[0]
print(wall.is_a()) # Returns 'IfcWall'

In [7]:
wall = ifc.by_type('IfcWall')[0]
print(wall.is_a()) # Returns 'IfcWall'

IfcWallStandardCase


You can also test if it is a certain class, as well as check for parent classes too:

In [8]:
print(wall.is_a('IfcWall')) # Returns True
print(wall.is_a('IfcElement')) # Returns True
print(wall.is_a('IfcWindow')) # Returns False

True
True
False


Let's quickly check the STEP ID of our element:

In [9]:
print(wall.id())

4131


Let's get some attributes of an element. IFC attributes have a particular order. We can access it just like a list, so let's get the first and third attribute:

In [10]:
print(wall[0]) # The first attribute is the GlobalId
print(wall[2]) # The third attribute is the Name

2O2Fr$t4X7Zf8NOew3FNld
Basic Wall:Interior - Partition (92mm Stud):138584


Knowing the order of attributes is boring and technical. We can access them by name too:

In [11]:
print(wall.GlobalId)
print(wall.Name)

2O2Fr$t4X7Zf8NOew3FNld
Basic Wall:Interior - Partition (92mm Stud):138584


Getting attributes one by one is tedious. Let's grab them all:

In [12]:
print(wall.get_info()) # Gives us a dictionary of attributes, such as {'id': 8, 'type': 'IfcWall', 'GlobalId': '2_qMTAIHrEYu0vYcqK8cBX', ... }

{'id': 4131, 'type': 'IfcWallStandardCase', 'GlobalId': '2O2Fr$t4X7Zf8NOew3FNld', 'OwnerHistory': #33=IfcOwnerHistory(#32,#2,$,.NOCHANGE.,$,$,$,0), 'Name': 'Basic Wall:Interior - Partition (92mm Stud):138584', 'Description': None, 'ObjectType': 'Basic Wall:Interior - Partition (92mm Stud):128360', 'ObjectPlacement': #4118=IfcLocalPlacement(#38,#4117), 'Representation': #4130=IfcProductDefinitionShape($,$,(#4121,#4129)), 'Tag': '138584'}


In [13]:
import ifcopenshell.util
import ifcopenshell.util.element
print(ifcopenshell.util.element.get_psets(wall))

{'Pset_WallCommon': {'Reference': 'Basic Wall:Interior - Partition (92mm Stud)', 'LoadBearing': False, 'ExtendToStructure': False, 'IsExternal': False}, 'PSet_Revit_Constraints': {'Location Line': 2, 'Base Constraint': 'Level 1', 'Base Offset': 0.0, 'Base is Attached': False, 'Base Extension Distance': 0.0, 'Top Constraint': 'Up to level: Level 2', 'Unconnected Height': 2.795000000000378, 'Top Offset': -0.3050000000000001, 'Top is Attached': False, 'Top Extension Distance': 0.0, 'Room Bounding': True, 'Related to Mass': False}, 'PSet_Revit_Other': {'InstallationDate': 'InstallationDate', 'SerialNumber': 'SerialNumber', 'WarrantyStartDate': 'WarrantyStartDate', 'BarCode': 'BarCode', 'AssetIdentifier': 'AssetIdentifier', 'TagNumber': 'TagNumber'}, 'PSet_Revit_Phasing': {'Phase Created': 'New Construction'}, 'PSet_Revit_Structural': {'Structural Usage': 0}, 'PSet_Revit_Dimensions': {'Length': 3.791499999999996, 'Area': 10.01448500000069, 'Volume': 1.241796140000085}, 'PSet_Revit_Type_Cons

Some attributes are special, called "inverse attributes". They happen when another element is referencing our element. They can reference it for many reasons, like to define a relationship, such as if they create a void in our wall, join our wall, or define a quantity take-off value for our wall, among others. Just treat them like regular attributes:

In [14]:
print(wall.IsDefinedBy)

(#4135=IfcRelDefinesByProperties('0wjzFZTLX6HAmUbkSg5xMg',#33,$,$,(#4131),#4134), #4186=IfcRelDefinesByProperties('32QEQ3DsT4_wd_$r05AO8y',#33,$,$,(#4131),#4185), #4188=IfcRelDefinesByProperties('1FWy0diTv5M9NoFfMP67OK',#33,$,$,(#4131),#4187), #4190=IfcRelDefinesByProperties('3NNuYaHqT80QOfSp_k9gZ7',#33,$,$,(#4131),#4189), #4192=IfcRelDefinesByProperties('0u6FYoK11ACeOwLbYm7bcm',#33,$,$,(#4131),#4191), #4194=IfcRelDefinesByProperties('08GWwNmmLCEOt0pcgpoO03',#33,$,$,(#4131),#4193), #38538=IfcRelDefinesByProperties('0CpuJJ7wTFA8vmntAxzlqi',#33,$,$,(#4131,#4219,#4508,#4553,#4598,#5642,#5687,#5731,#5859,#5903,#5948,#5992,#6036,#24723,#35199,#35357,#35452,#35497),#4195), #38539=IfcRelDefinesByProperties('15WoIPLkr0IA1HmacUbJIG',#33,$,$,(#4131,#4219,#4508,#4553,#4598,#5642,#5687,#5731,#5859,#5903,#5948,#5992,#6036,#24723,#35199,#35357,#35452,#35497),#4196), #38540=IfcRelDefinesByProperties('1W4LP$v3X95fgY$se$JZOz',#33,$,$,(#4131,#4219,#4508,#4553,#4598,#5642,#5687,#5731,#5859,#5903,#5948,#5

Perhaps we want to see all elements which are referencing our wall?

In [15]:
print(ifc.get_inverse(wall))

[#4135=IfcRelDefinesByProperties('0wjzFZTLX6HAmUbkSg5xMg',#33,$,$,(#4131),#4134), #4186=IfcRelDefinesByProperties('32QEQ3DsT4_wd_$r05AO8y',#33,$,$,(#4131),#4185), #4188=IfcRelDefinesByProperties('1FWy0diTv5M9NoFfMP67OK',#33,$,$,(#4131),#4187), #4190=IfcRelDefinesByProperties('3NNuYaHqT80QOfSp_k9gZ7',#33,$,$,(#4131),#4189), #4192=IfcRelDefinesByProperties('0u6FYoK11ACeOwLbYm7bcm',#33,$,$,(#4131),#4191), #4194=IfcRelDefinesByProperties('08GWwNmmLCEOt0pcgpoO03',#33,$,$,(#4131),#4193), #38298=IfcRelContainedInSpatialStructure('1OnF7j$Fj2cP4Xy3mK4kIj',#33,$,$,(#3797,#3999,#4043,#4087,#4131,#4219,#4287,#4399,#4465,#4508,#4553,#4598,#5165,#5267,#5642,#5903,#6426,#6531,#6652,#6757,#6921,#7025,#8066,#8169,#9021,#21401,#21449,#21497,#21545,#21610,#21658,#21821,#21929,#23671,#23768,#23826,#23884,#23992,#24596,#24680,#24723,#24768,#24813,#32116,#35452,#35497,#36686,#36892,#37325,#37456,#37777,#37864),#39), #38360=IfcRelAssociatesMaterial('0ULqLytwDBjhSuu72Oe7Pr',#33,$,$,(#4131),#4203), #38538=IfcR

Let's do the opposite, let's see all the elements which our wall references instead:

In [16]:
print(ifc.traverse(wall))

[#4131=IfcWallStandardCase('2O2Fr$t4X7Zf8NOew3FNld',#33,'Basic Wall:Interior - Partition (92mm Stud):138584',$,'Basic Wall:Interior - Partition (92mm Stud):128360',#4118,#4130,'138584'), #33=IfcOwnerHistory(#32,#2,$,.NOCHANGE.,$,$,$,0), #32=IfcPersonAndOrganization(#30,#31,$), #30=IfcPerson($,$,'cskender',$,$,$,$,$), #31=IfcOrganization($,'','',$,$), #2=IfcApplication(#1,'2011','Autodesk Revit Architecture 2011','Revit'), #1=IfcOrganization($,'Autodesk Revit Architecture 2011',$,$,$), #4118=IfcLocalPlacement(#38,#4117), #38=IfcLocalPlacement(#25,#37), #25=IfcLocalPlacement(#38273,#24), #38273=IfcLocalPlacement($,#38272), #38272=IfcAxis2Placement3D(#3,$,$), #3=IfcCartesianPoint((0.,0.,0.)), #24=IfcAxis2Placement3D(#3,$,$), #37=IfcAxis2Placement3D(#3,$,$), #4117=IfcAxis2Placement3D(#4116,#9,#8), #4116=IfcCartesianPoint((2.538,-0.4170000000000082,0.)), #9=IfcDirection((0.,0.,1.)), #8=IfcDirection((0.,-1.,0.)), #4130=IfcProductDefinitionShape($,$,(#4121,#4129)), #4121=IfcShapeRepresentatio

In [17]:
print(ifc.traverse(wall, max_levels=1)) # Or, let's just go down one level deep

[#4131=IfcWallStandardCase('2O2Fr$t4X7Zf8NOew3FNld',#33,'Basic Wall:Interior - Partition (92mm Stud):138584',$,'Basic Wall:Interior - Partition (92mm Stud):128360',#4118,#4130,'138584'), #33=IfcOwnerHistory(#32,#2,$,.NOCHANGE.,$,$,$,0), #4118=IfcLocalPlacement(#38,#4117), #4130=IfcProductDefinitionShape($,$,(#4121,#4129))]


If you want to modify data, just assign it to the relevant attribute:

In [18]:
wall.Name = 'My new wall name'

You can also generate new GlobalIds:

In [19]:
wall.GlobalId = ifcopenshell.guid.new()

After modifying some IFC data, you can save it to a new IFC-SPF file:

#ifc.write('/path/to/a/new.ifc')

# Writing IFC

You can generate a new IFC from scratch too, instead of reading an existing one:

In [20]:
#ifc = ifcopenshell.file()
# Or if you want a particular schema:
ifc = ifcopenshell.file(schema='IFC4')

You can create new IFC elements, and add it either to an existing or newly created IFC file object:

In [21]:
new_wall = ifc.createIfcWall() # Will return #1=IfcWall($,$,$,$,$,$,$,$,$) - notice all of the attributes are blank!
print(ifc.by_type('IfcWall')) # Will return a list with our wall in it: [#1=IfcWall($,$,$,$,$,$,$,$,$)]

[#1=IfcWall($,$,$,$,$,$,$,$,$)]


Alternatively, you can also use this way to create new elements:

In [22]:
ifc.create_entity('IfcWall')

#2=IfcWall($,$,$,$,$,$,$,$,$)

Specifying more arguments lets you fill in attributes while creating the element instead of assigning them separately. You specify them in the order of the attributes.

In [23]:
ifc.create_entity('IfcWall', ifcopenshell.guid.new()) # Gives us #1=IfcWall('0EI0MSHbX9gg8Fxwar7lL8',$,$,$,$,$,$,$,$)

#3=IfcWall('1aknkbxZvEiusd_A3Iz70m',$,$,$,$,$,$,$,$)

Again, knowing the order of attributes is difficult, so you can use keyword arguments instead:

In [24]:
ifc.create_entity('IfcWall', GlobalId=ifcopenshell.guid.new(), Name='Wall Name') # Gives us #1=IfcWall('0EI0MSHbX9gg8Fxwar7lL8',$,'Wall Name',$,$,$,$,$,$)

#4=IfcWall('1fTqQr2AX2cugLWX7pIVw8',$,'Wall Name',$,$,$,$,$,$)

Sometimes, it's easier to expand a dictionary:

In [25]:
data = {
    'GlobalId': ifcopenshell.guid.new(),
    'Name': 'Wall Name'
}
ifc.create_entity('IfcWall', **data)

#5=IfcWall('14074mHgn4I83eipnRgo8R',$,'Wall Name',$,$,$,$,$,$)

Some attributes of an element aren't just text, they may be a reference to another element. Easy:

In [26]:
wall = ifc.createIfcWall()
wall.OwnerHistory = ifc.createIfcOwnerHistory()

What if we already have an element from one IFC file and want to add it to another?

In [36]:
wall = ifc.by_type('IfcWall')[0]
new_ifc = ifcopenshell.file(schema='IFC4')
new_ifc.add(wall)


#1=IfcWall($,$,$,$,$,$,$,$,$)

Fed up with an object? Let's delete it:

In [38]:
#new_ifc.remove(wall) #this isnt working...

# Geometrey

The usage of IfcOpenShell for geometry processing is currently considered to be moderate to advanced. There are two approaches to processing geometry. One approach is to traverse the Representation attribute of the IFC element, and parse it yourself. This requires an in-depth understanding of IFC geometric representations, as well as its many caveats with units and transformations, but can be very simple to extract specific types of geometry. The second approach is to use IfcOpenShell's shape processing features, which will convert almost all IFC representations into a triangulated mesh. Regardless of the source format, once it is in a mesh representation, you may use standard mesh geometry processing algorithms to analyse the geometry. This makes it easier to write generic code for any representation, but may be harder to extract certain geometric features.

In [4]:
import multiprocessing
import ifcopenshell
import ifcopenshell.geom

ifcPath="../data/Duplex_A.ifc"

try:
    ifc_file = ifcopenshell.open("../data/Duplex_A.ifc")
except:
    print(ifcopenshell.get_log())
else:
    settings = ifcopenshell.geom.settings()
    iterator = ifcopenshell.geom.iterator(settings, ifc_file, multiprocessing.cpu_count())
    count = 0
    if iterator.initialize():
        while iterator.next():
            count = count +1
            shape = iterator.get()
            element = ifc_file.by_guid(shape.guid)
            faces = shape.geometry.faces # Indices of vertices per triangle face e.g. [f1v1, f1v2, f1v3, f2v1, f2v2, f2v3, ...]
            verts = shape.geometry.verts # X Y Z of vertices in flattened list e.g. [v1x, v1y, v1z, v2x, v2y, v2z, ...]
            materials = shape.geometry.materials # Material names and colour style information that are relevant to this shape
            material_ids = shape.geometry.material_ids # Indices of material applied per triangle face e.g. [f1m, f2m, ...]

            # Since the lists are flattened, you may prefer to group them per face like so depending on your geometry kernel
            grouped_verts = [[verts[i], verts[i + 1], verts[i + 2]] for i in range(0, len(verts), 3)]
            grouped_faces = [[faces[i], faces[i + 1], faces[i + 2]] for i in range(0, len(faces), 3)]
    
    print('went over {} shapes'.format(count))

print('verts:')
print(grouped_verts)
print('faces:')
print(grouped_faces)

went over 285 shapes
verts:
[[1.1350999999999587, 0.02700000000000009, 0.0], [1.1350999999999587, 0.02700000000000009, 0.2880000000000125], [-2.220446049250313e-16, 0.02700000000000009, 0.2880000000000125], [-2.220446049250313e-16, 0.02700000000000009, 0.0], [-2.220446049250313e-16, -0.02700000000000009, 0.2880000000000125], [-2.220446049250313e-16, -0.02700000000000009, 0.0], [1.1350999999999587, -0.02700000000000009, 0.2880000000000125], [1.1350999999999587, -0.02700000000000009, 0.0]]
faces:
[[1, 0, 3], [2, 1, 3], [2, 3, 5], [4, 2, 5], [4, 5, 7], [6, 4, 7], [6, 7, 0], [1, 6, 0], [5, 3, 0], [7, 5, 0], [1, 2, 4], [1, 4, 6]]
